In [1]:
import duckdb
import pickle
import os

In [2]:
%load_ext sql

In [4]:
file_path = "/Users/nikola/Downloads/publications.pkl"

# Specify the file path to persist the database
database_file = 'publications.db'

def add_primary_key(con, table_name, column_name):
    con.execute(f"CREATE UNIQUE INDEX ON {table_name} ({column_name});")

def load_publications(file_path):
    try:
        publications = []
        with open(file_path, 'rb') as file:
            while True:
                try:
                    batch = pickle.load(file)
                    publications.extend(batch)
                except EOFError:
                    break
        return publications
    except IOError as e:
        print(f"Error: {str(e)}")
        return None

def create_duckdb_table():
    try:
        # Check if the database file already exists
        if os.path.exists(database_file):
            con = duckdb.connect(database=database_file)
            print("Using existing database file.")
        else:
            con = duckdb.connect(database=database_file)
            print("Creating new database file.")

        # Check if the table already exists
        con.execute("SHOW TABLES;")
        tables = [row[0] for row in con.fetchall()]
        if "publications" not in tables:
            publications = load_publications(file_path)

            # Get the column names from the first publication
            columns = list(publications[0].keys())

            # Create the CREATE TABLE statement dynamically
            create_table_stmt = "CREATE TABLE publications ("
            create_table_stmt += ", ".join([f"{col} VARCHAR" for col in columns])
            create_table_stmt += ")"

            con.execute(create_table_stmt)

            # Insert the publications data into the table
            for publication in publications:
                values = [str(publication.get(col, '')) for col in columns]
                insert_stmt = f"INSERT INTO publications VALUES ({','.join(['?' for _ in values])})"
                con.execute(insert_stmt, values)

            # Add a primary key constraint to the table
            add_primary_key(con, "publications", "crosbiId")

        return con
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

con = create_duckdb_table()
if con is not None:
    print("DuckDB table created or loaded successfully.")
else:
    print("Failed to create or load DuckDB table.")

Creating new database file.
Error: Not implemented Error: Please provide an index name, e.g., CREATE INDEX my_name ...
Failed to create or load DuckDB table.


In [9]:
con = duckdb.connect(database='publications.db')

In [21]:
con.execute(f"CREATE INDEX publications_crosbiid_idx ON publications (crosbiId);")

In [20]:
%%sql
SELECT *
FROM publications
WHERE crosbiId = 795775;

Running query in 'duckdb:///publications.db'

crosbiId,bibIrbId,autori,naslov,vrsta,tip,godina,nadredenaPublikacija,casopis,volumen,svescic,stranice,doi,issn,indeksiranost,citati,naslovi,sazeci,kljucneRijeci,poveznice,osobeResources,ustanoveResources,_links
795775,,,Solid-State Science & Research Book of Abstracts and Programme 2023,urednička knjiga,zbornik sažetaka s konferencije,2023,Mechanochemical synthesis of some transition metal polycyanometallates,,,,,,,,"[{'vrstaCitataId': 1040, 'vrstaCitataNaziv': 'CROSBI', 'citat': 'Solid-State Science & Research Book of Abstracts and Programme 2023 / Biliškov, Nikola; Karadeniz, Bahar; Juraj Pantalon, Natalija (ur.) \nZagreb: Institut Ruđer Bošković, 2023'}]","[{'cfLangCode': 'en', 'naziv': 'Solid-State Science & Research Book of Abstracts and Programme 2023', 'original': True, 'cfTrans': 'o'}]","[{'cfLangCode': 'en', 'naziv': 'Book of abstracts and programme for the Solid-State Science & Research conference held 28-30 June 2023', 'original': True, 'cfTrans': 'o'}]","[{'cfLangCode': 'en', 'naziv': 'solid ; state ; science ; book ; abstracts ; 2023', 'original': True, 'cfTrans': 'o'}]","[{'urlId': 235635, 'urlVrstaId': 990, 'urlVrstaNaziv': 'URL rada koji je dostupan u otvorenom pristupu', 'url': 'https://drive.google.com/file/d/1Cp1XNoxnVxPTpc-7EX0pLmUyipgW-Z34/view'}]","{'_embedded': {'osobe': [{'crorisId': 34243, 'titulaIspredImena': 'dr. sc.', 'ime': 'Bahar', 'prezime': 'Karadeniz', 'funkcija': {'id': 906, 'naziv': 'urednik/ci'}, '_links': {'self': {'href': 'https://croris.hr/crosbi-api/osoba/34243'}}}, {'crorisId': 2422, 'titulaIspredImena': 'dr. sc.', 'ime': 'Nikola', 'prezime': 'Biliškov', 'funkcija': {'id': 906, 'naziv': 'urednik/ci'}, '_links': {'self': {'href': 'https://croris.hr/crosbi-api/osoba/2422'}}}]}}","{'_embedded': {'ustanove': [{'crorisId': 66, 'naziv': 'Institut Ruđer Bošković', 'mbu': 98, 'funkcija': {'id': 955, 'naziv': 'ustanova urednika'}, '_links': {'self': {'href': 'https://croris.hr/crosbi-api/ustanova/66'}}}]}}","{'osobe': {'href': 'https://croris.hr/crosbi-api/osoba/publikacija/795775'}, 'ustanove': {'href': 'https://croris.hr/crosbi-api/ustanova/publikacija/795775'}, 'self': {'href': 'https://croris.hr/crosbi-api/publikacija/795775'}}"
795775,,,Solid-State Science & Research Book of Abstracts and Programme 2023,urednička knjiga,zbornik sažetaka s konferencije,2023,Advantages of crosslinking self-assembled molecular layers on metal surfaces,,,,,,,,"[{'vrstaCitataId': 1040, 'vrstaCitataNaziv': 'CROSBI', 'citat': 'Solid-State Science & Research Book of Abstracts and Programme 2023 / Biliškov, Nikola; Karadeniz, Bahar; Juraj Pantalon, Natalija (ur.) \nZagreb: Institut Ruđer Bošković, 2023'}]","[{'cfLangCode': 'en', 'naziv': 'Solid-State Science & Research Book of Abstracts and Programme 2023', 'original': True, 'cfTrans': 'o'}]","[{'cfLangCode': 'en', 'naziv': 'Book of abstracts and programme for the Solid-State Science & Research conference held 28-30 June 2023', 'original': True, 'cfTrans': 'o'}]","[{'cfLangCode': 'en', 'naziv': 'solid ; state ; science ; book ; abstracts ; 2023', 'original': True, 'cfTrans': 'o'}]","[{'urlId': 235635, 'urlVrstaId': 990, 'urlVrstaNaziv': 'URL rada koji je dostupan u otvorenom pristupu', 'url': 'https://drive.google.com/file/d/1Cp1XNoxnVxPTpc-7EX0pLmUyipgW-Z34/view'}]","{'_embedded': {'osobe': [{'crorisId': 34243, 'titulaIspredImena': 'dr. sc.', 'ime': 'Bahar', 'prezime': 'Karadeniz', 'funkcija': {'id': 906, 'naziv': 'urednik/ci'}, '_links': {'self': {'href': 'https://croris.hr/crosbi-api/osoba/34243'}}}, {'crorisId': 2422, 'titulaIspredImena': 'dr. sc.', 'ime': 'Nikola', 'prezime': 'Biliškov', 'funkcija': {'id': 906, 'naziv': 'urednik/ci'}, '_links': {'self': {'href': 'https://croris.hr/crosbi-api/osoba/2422'}}}]}}","{'_embedded': {'ustanove': [{'crorisId': 66, 'naziv': 'Institut Ruđer Bošković', 'mbu': 98, 'funkcija': {'id': 955, 'naziv': 'ustanova urednika'}, '_links': {'self': {'href': 'https://croris.hr/crosbi-api/ustanova/66'}}}]}}","{'osobe': {'href': 'https:/

In [22]:
con.execute("SHOW publications")

In [58]:
# Print the table structure
table_info = con.execute("PRAGMA table_info('publications')").fetchall()
print("Table structure:")
for column in table_info:
    print(f"Column: {column[1]}, Type: {column[2]}")

Table structure:
Column: crosbiId, Type: VARCHAR
Column: bibIrbId, Type: VARCHAR
Column: autori, Type: VARCHAR
Column: naslov, Type: VARCHAR
Column: vrsta, Type: VARCHAR
Column: tip, Type: VARCHAR
Column: godina, Type: VARCHAR
Column: nadredenaPublikacija, Type: VARCHAR
Column: casopis, Type: VARCHAR
Column: volumen, Type: VARCHAR
Column: svescic, Type: VARCHAR
Column: stranice, Type: VARCHAR
Column: doi, Type: VARCHAR
Column: issn, Type: VARCHAR
Column: indeksiranost, Type: VARCHAR
Column: citati, Type: VARCHAR
Column: naslovi, Type: VARCHAR
Column: sazeci, Type: VARCHAR
Column: kljucneRijeci, Type: VARCHAR
Column: poveznice, Type: VARCHAR
Column: osobeResources, Type: VARCHAR
Column: ustanoveResources, Type: VARCHAR
Column: _links, Type: VARCHAR


In [12]:
%sql duckdb:///publications.db

Connecting to 'duckdb:///publications.db'

In [23]:
%%sql
SELECT COUNT(*)
FROM publications
WHERE REGEXP_MATCHES(autori, '(Ezgeta[- ]?Bali[cć]?[,]? D(.|aria))|(D(.|aria)[,]? Ezgeta[- ]?Bali[cć]?)|(Ezgeta[- ]?D(.|aria))|(D(.|aria)[,]? Ezgeta)')

Running query in 'duckdb:///publications.db'

count_star()
70


In [23]:
# Print one row from the table
row = con.execute("SELECT * FROM publications LIMIT 1").fetchone()
print("Sample row:")
for column, value in zip(table_info, row):
    print(f"{column[1]}: {value}")

Sample row:
crosbiId: 304137
bibIrbId: 1172097
autori: Splieth, Christian H. ; Banerjee, Avijit ; Bottenberg, Peter ; Breschi, Lorenzo ; Campus, Guglielmo ; Ekstrand, Kim Rud ; Giacaman, Rodrigo A. ; Haak, Rainer ; Hannig, Matthias ; Hickel, Reinhard ; Jurić, Hrvoje ; Lussi, Adrian ; Machiulskiene, Vita ; Manton, David J. ; Jablonski-Momeni, Anahita ; Opdam, Niek J.M. ; Paris, Sebastian ; Santamaría, Ruth M. ; Schwendicke, Falk ; Tassery, Herve ; Ferreira Zandona, Andrea ; Zero, Domenick T. ; Zimmer, Stefan ; Doméjean, Sophie
naslov: How to Intervene in the Caries Process in Children: A Joint ORCA and EFCD Expert Delphi Consensus Statement
vrsta: prilog u časopisu
tip: izvorni znanstveni rad
godina: 2020
nadredenaPublikacija: Caries research
casopis: Caries research
volumen: 54
svescic: 4
stranice: str. 297-305
doi: 10.1159/000507692
issn: 0008-6568
indeksiranost: ['Scopus', 'Current Contents Connect (CCC)', 'Medline', 'Web of Science Core Collection, Science Citation Index Expanded (W

In [24]:
import json

# Create a separate table for persons
con.execute("""
    CREATE TABLE osobe (
        crorisId INTEGER PRIMARY KEY,
        titulaIspredImena VARCHAR,
        ime VARCHAR,
        prezime VARCHAR
    )
""")

# Create a table to establish the relationship between persons and publications
con.execute("""
    CREATE TABLE osobe_publikacije (
        crorisId INTEGER,
        crosbiId INTEGER,
        funkcija VARCHAR,
        PRIMARY KEY (crorisId, crosbiId),
        FOREIGN KEY (crorisId) REFERENCES osobe (crorisId),
        FOREIGN KEY (crosbiId) REFERENCES publications (crosbiId)
    )
""")

# Iterate over each row in the "publications" table
rows = con.execute("SELECT crosbiId, osobeResources FROM publications").fetchall()
for row in rows:
    crosbiId = row[0]
    osobeResources = json.loads(row[1])

    if '_embedded' in osobeResources and 'osobe' in osobeResources['_embedded']:
        osobe = osobeResources['_embedded']['osobe']
        for osoba in osobe:
            crorisId = osoba['crorisId']
            titulaIspredImena = osoba.get('titulaIspredImena', '')
            ime = osoba['ime']
            prezime = osoba['prezime']
            funkcija = osoba['funkcija']['naziv']

            # Insert the person into the "osobe" table if it doesn't exist
            con.execute("""
                INSERT OR IGNORE INTO osobe (crorisId, titulaIspredImena, ime, prezime)
                VALUES (?, ?, ?, ?)
            """, (crorisId, titulaIspredImena, ime, prezime))

            # Insert the relationship between the person and publication into the "osobe_publikacije" table
            con.execute("""
                INSERT INTO osobe_publikacije (crorisId, crosbiId, funkcija)
                VALUES (?, ?, ?)
            """, (crorisId, crosbiId, funkcija))

# Print the contents of the "osobe" table
print("Osobe:")
osobe = con.execute("SELECT * FROM osobe").fetchall()
for osoba in osobe:
    print(osoba)

# Print the contents of the "osobe_publikacije" table
print("\nOsobe-Publikacije:")
osobe_publikacije = con.execute("SELECT * FROM osobe_publikacije").fetchall()
for veza in osobe_publikacije:
    print(veza)

BinderException: Binder Error: Failed to create foreign key: there is no primary key or unique constraint for referenced table "publications"

In [11]:
# Filtering for rows where 'tip' is 'izvorni znanstveni rad'
filtered_df = con.execute("SELECT * FROM publications WHERE tip = 'izvorni znanstveni rad'").df()
print(f"Filtered DataFrame showing only 'izvorni znanstveni rad': {len(filtered_df)}")

Filtered DataFrame showing only 'izvorni znanstveni rad': 288969


In [20]:
# Applying the condition to filter rows from the year 2004 to 2023 inclusive
filtered_df2 = con.execute("SELECT * FROM publications WHERE CAST(godina AS INTEGER) BETWEEN 2004 AND 2023 AND tip = 'izvorni znanstveni rad'").df()
print(f"Number of rows from 2004 to 2023 showing only 'izvorni znanstveni rad': {len(filtered_df2)}")

Number of rows from 2004 to 2023 showing only 'izvorni znanstveni rad': 223534


In [21]:
# Filtering for rows where 'tip' is 'izvorni znanstveni rad'
filtered_count = con.execute("SELECT COUNT(*) FROM publications WHERE CAST(godina AS INTEGER) BETWEEN 2004 AND 2023 AND tip = 'izvorni znanstveni rad'").fetchone()[0]
print(f"Number of rows from 2004 to 2023 showing only 'izvorni znanstveni rad': {filtered_count}")

Number of rows from 2004 to 2023 showing only 'izvorni znanstveni rad': 223534
